In [11]:
import os
import cv2
import torch
import numpy
import torch.nn.functional as F

from PIL import Image
from torchvision import transforms
from torch.autograd import Variable
from collections import OrderedDict

SAVE_PATH = "./results/"


IS_MULTISCALE = True
N_CLASS = 19
COLOR_MAP = [(128, 64, 128), (244, 35, 232), (70, 70, 70), (102, 102, 156), (190, 153, 153), (153, 153, 153),
             (250, 170, 30), (220, 220, 0), (107, 142, 35), (152, 251, 152), (70, 130, 180), (220, 20, 60),
             (255,  0,  0), (0, 0, 142), (0, 0, 70), (0, 60, 100), (0, 80, 100), (0, 0, 230), (119, 11, 32)]

inf_scales = [0.5, 0.75, 1.0, 1.25, 1.5, 1.8]
data_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize([0.290101, 0.328081, 0.286964],
                                                           [0.182954, 0.186566, 0.184475])])


class Inference(object):

    def __init__(self, model_name, model_path):
        print(model_name,model_path)

        self.seg_model = self.__init_model(model_name, model_path, is_local=False)

    def __init_model(self, model_name, model_path, is_local=False):
        if model_name == 'MobileNetDenseASPP':
            from cfgs.MobileNetDenseASPP import Model_CFG
            from models.MobileNetDenseASPP import DenseASPP
        elif model_name == 'DenseASPP121':
            from cfgs.DenseASPP121 import Model_CFG
            from models.DenseASPP import DenseASPP
        elif model_name == 'DenseASPP169':
            from cfgs.DenseASPP169 import Model_CFG
            from models.DenseASPP import DenseASPP
        elif model_name == 'DenseASPP201':
            from cfgs.DenseASPP201 import Model_CFG
            from models.DenseASPP import DenseASPP
        elif model_name == 'DenseASPP161':
            from cfgs.DenseASPP161 import Model_CFG
            from models.DenseASPP import DenseASPP
        else:
            from cfgs.DenseASPP161 import Model_CFG
            from models.DenseASPP import DenseASPP

        seg_model = DenseASPP(Model_CFG, n_class=N_CLASS, output_stride=8)
        #self.__load_weight(seg_model, model_path, is_local=is_local)
        seg_model.train()
        seg_model = seg_model.cuda()

        return seg_model

    def folder_inference(self, img_dir, is_multiscale=True):
        folders = sorted(os.listdir(img_dir))
        for f in folders:
            save_path = SAVE_PATH + f + "/"
            read_path = os.path.join(img_dir, f)
            names = sorted(os.listdir(read_path))
            for n in names:
                if not n.endswith(".png"):
                    continue
                print(n)
                read_name = os.path.join(read_path, n)
                img = Image.open(read_name)
                if is_multiscale:
                    pre = self.multiscale_inference(img)
                else:
                    pre = self.single_inference(img)
                mask = self.pre_to_img(pre)
                cv2.imwrite(save_path + n, mask)
                print(save_path + n)
                #cv2.imshow('DenseASPP', mask)
                #cv2.waitKey(0)

    def multiscale_inference(self, test_img):
        h, w = test_img.size
        pre = []
        for scale in inf_scales:
            img_scaled = test_img.resize((int(h * scale), int(w * scale)), Image.CUBIC)
            pre_scaled = self.single_inference(img_scaled, is_flip=False)
            pre.append(pre_scaled)

            img_scaled = img_scaled.transpose(Image.FLIP_LEFT_RIGHT)
            pre_scaled = self.single_inference(img_scaled, is_flip=True)
            pre.append(pre_scaled)

        pre_final = self.__fushion_avg(pre)

        return pre_final

    def single_inference(self, test_img, is_flip=False):
        with torch.no_grad():
            image = Variable(data_transforms(test_img).unsqueeze(0).cuda())
            pre = self.seg_model.forward(image)
            print(pre.shape)

            if pre.size()[0] < 1024:
                pre = F.upsample(pre, size=(1024, 2048), mode='bilinear')

            pre = F.log_softmax(pre, dim=1)
            pre = pre.data.cpu().numpy()

            if is_flip:
                tem = pre[0]
                tem = tem.transpose(1, 2, 0)
                tem = numpy.fliplr(tem)
                tem = tem.transpose(2, 0, 1)
                pre[0] = tem

            return pre

    @staticmethod
    def __fushion_avg(pre):
        pre_final = 0
        for pre_scaled in pre:
            pre_final = pre_final + pre_scaled
        pre_final = pre_final / len(pre)
        return pre_final

    @staticmethod
    def __load_weight(seg_model, model_path, is_local=True):
        print("loading pre-trained weight")
        weight = torch.load(model_path, map_location=lambda storage, loc: storage)

        if is_local:
            seg_model.load_state_dict(weight)
        else:
            new_state_dict = OrderedDict()
            for k, v in weight.items():
                name = k[7:]  # remove `module.`
                new_state_dict[name] = v
            seg_model.load_state_dict(new_state_dict)
        print("loading complete")

    @staticmethod
    def pre_to_img(pre):
        result = pre.argmax(axis=1)[0]
        row, col = result.shape
        dst = numpy.zeros((row, col, 3), dtype=numpy.uint8)
        for i in range(N_CLASS):
            dst[result == i] = COLOR_MAP[i]
        dst = numpy.array(dst, dtype=numpy.uint8)
        dst = cv2.cvtColor(dst, cv2.COLOR_RGB2BGR)
        return dst

In [12]:
import pickle as pickle

model_path = './weights/denseASPP161.pkl'

fr = open(model_path,'rb')    #open的参数是pkl文件的路径
inf = pickle.load(fr)       #读取pkl文件的内容
print(inf)
fr.close()                       #关闭文件


119547037146038801333356


In [13]:
torch.cuda.set_device(0)
model_name = 'DenseASPP161'
model_path = './weights/denseASPP161.pkl'

infer = Inference(model_name, model_path)

DenseASPP161 ./weights/denseASPP161.pkl


In [14]:
img_dir = './data'
infer.folder_inference(img_dir, is_multiscale=False)
infer.folder_inference(img_dir, is_multiscale=False)

aachen_000000_000019_leftImg8bit.png
torch.Size([1, 19, 1024, 2048])
./results/data/aachen_000000_000019_leftImg8bit.png
微信截图_20190426111814.png
torch.Size([1, 19, 584, 864])
./results/data/微信截图_20190426111814.png


NotADirectoryError: [Errno 20] Not a directory: './data/data.zip'

In [44]:
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import os

def train(epoch, infer, lossFunction, optimizer, device, trainloader):

    print('\nEpoch: %d' % epoch)
    infer.seg_model.train()     # enter train mode
    train_loss = 0    # accumulate every batch loss in a epoch
    correct = 0       # count when model' prediction is correct i train set
    total = 0         # total number of prediction in train set
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device) # load data to gpu device
        inputs, targets = Variable(inputs), Variable(targets)
        optimizer.zero_grad()            # clear gradients of all optimized torch.Tensors'
        pre = infer.seg_model.forward(inputs)
        print(pre.shape)

        if pre.size()[0] < 1024:
            pre = F.upsample(pre, size=(1024, 2048), mode='bilinear')

        pre = F.log_softmax(pre, dim=1)
        pre = pre.data.cpu().numpy()

        outputs = Inference.pre_to_img(pre)          # forward propagation return the value of softmax function
        print(inputs.shape,outputs.shape,targets.shape)
        loss = lossFunction(outputs, targets) #compute loss
        loss.backward()                  # compute gradient of loss over parameters 
        optimizer.step()                 # update parameters with gradient descent 

        train_loss += loss.item()        # accumulate every batch loss in a epoch
        _, predicted = outputs.max(1)    # make prediction according to the outputs
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item() # count how many predictions is correct
        
        if (batch_idx+1) % 100 == 0:
            # print loss and acc
            print( 'Train loss: %.3f | Train Acc: %.3f%% (%d/%d)'
                % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    print( 'Train loss: %.3f | Train Acc: %.3f%% (%d/%d)'
                % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    
    
def test(model, lossFunction, optimizer, device, testloader):
    """
    test model's prediction performance on loader.  
    When thid function is called, model is evaluated.
    Args:
        loader: data for evaluation
        model: prediction model
        loss_fn: loss function to judge the distance between target and outputs
    output:
        total_loss
        accuracy
    """
    global best_acc
    model.eval() #enter test mode
    test_loss = 0 # accumulate every batch loss in a epoch
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = lossFunction(outputs, targets) #compute loss

            test_loss += loss.item() # accumulate every batch loss in a epoch
            _, predicted = outputs.max(1) # make prediction according to the outputs
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item() # count how many predictions is correct
        # print loss and acc
        print('Test Loss: %.3f  | Test Acc: %.3f%% (%d/%d)'
            % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

        
def data_loader():
    # define method of preprocessing data for evaluating
    transform_train = transforms.Compose([
        transforms.Resize(32),
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # Normalize a tensor image with mean and standard variance
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor(),
        # Normalize a tensor image with mean and standard variance
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    
    # prepare dataset by ImageFolder, data should be classified by directory
    trainset = torchvision.datasets.ImageFolder(root='./mnist/train', transform=transform_train)

    testset = torchvision.datasets.ImageFolder(root='./mnist/test', transform=transform_test)

    # Data loader. 

    # Combines a dataset and a sampler, 

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)
    return trainloader, testloader

def run(infer, num_epochs):
    
    # load model into GPU device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    infer.seg_model.to(device)
    
    
    # define the loss function and optimizer

    lossFunction = nn.MSELoss()
    lr = 0.01
    optimizer = optim.SGD(infer.seg_model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    trainloader, testloader = data_loader()
    for epoch in range(num_epochs):
        train(epoch, infer, lossFunction, optimizer, device, trainloader)
        test(model, lossFunction, optimizer, device, testloader)
        if (epoch + 1) % 50 == 0 :
            lr = lr / 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

In [45]:
run(infer,num_epochs=20)


Epoch: 0
torch.Size([64, 19, 32, 32])
torch.Size([64, 3, 32, 32]) (32, 32, 3) torch.Size([64])


TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray